In [1]:
print("Kernel working fine!")

Kernel working fine!


## Adding basic imports

In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
829,Snakes on a Train is a movie I rented due to t...,negative
159,I think One True Thing is one of Meryl Streeps...,positive
869,"Fame, I think, was the best movie that I have ...",positive
135,This started out as a good sketch comedy. The ...,negative
678,"I was prepared for a bad movie, and a bad movi...",negative


## Data Pre-Processing

In [4]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
import nltk
nltk.download('wordnet')

df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/atharvabot7/nltk_data...


,review,sentiment
829,snake train movie rented due pure amusement th...,negative
159,think one true thing one meryl streep finest m...,positive
869,fame think best movie ever seen way funny dram...,positive
135,started good sketch comedy first show good loo...,negative
678,prepared bad movie bad movie guess complain tw...,negative


In [7]:
# Checking the number fo positive and negative sentiments for bias

df['sentiment'].value_counts()

sentiment
negative    257
positive    243
Name: count, dtype: int64

In [8]:
# Choosing only positive and negative sentiments, removing all others

x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'].value_counts()

sentiment
negative    257
positive    243
Name: count, dtype: int64

In [10]:
# Mapiing Positive sentiment to 1 and Negative to 0.

df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
829,snake train movie rented due pure amusement th...,0
159,think one true thing one meryl streep finest m...,1
869,fame think best movie ever seen way funny dram...,1
135,started good sketch comedy first show good loo...,0
678,prepared bad movie bad movie guess complain tw...,0


In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

## Train Test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/atharvachundurwar841/MLOps-Project.mlflow')
dagshub.init(repo_owner='atharvachundurwar841', repo_name='MLOps-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")